# Install and use DSE-DO-Utils in CPDv2.5
CPDv2.5 is very different from the previous versions and it has a significant impact on how the dse-do-utils can be installed and used.

Installation options:
* [1. Install using pip in a customized environment](#pip_install). This applies to both Jupyter and JupyterLab.
* [2. Install as a package in JupyterLab](#jl_install).
* [3. Install as modules in Jupyter](#jupyter_install).
* [4. Install as modules in the DO Model Builder](#mb_install).

Usage (import) options:
* [5. Import from the dse_do_utils package](#package_import).
* [6. Import from a module within the package](#submodule_import).
* [7. Import from a module as part of a set of modules](#module_import).

## 1. Install in customized environment<a class="anchor" id="pip_install"></a>
CPDv2.5 allows for easy customization of environments.
Add the following to the customization configuration:
```
- pip:
    - dse-do-utils=0.3.0.0
```
This automatically downloads dse-do-utils from PyPI and installs the package.

For air-gapped systems that have no access to PyPI (applies to any custom installed package):
1. Download the package from PyPI/Conda from an internet connected system as a wheel/zip file
2. Upload the wheel/zip as a data asset
3. Install package from wheel/zip

This downloads the package as a wheel/zip and puts it in the data assets
```
!pip download dse-do-utils -d /project_data/data_asset/
```
Then run the following to make the file visible in the WS Data Assets UI:
```
file_name = 'dse_do_utils-0.3.0.0.tar.gz'
file_path = '/project_data/data_asset/' + file_name
with open(file_path, 'rb') as f:
    from project_lib import Project
    project = Project.access()
    project.save_data(file_name=file_name, data=f, overwrite=True)
```
Once it is in the data assets, the project can be exported and moved to the air-gapped system

Next, set the environment customization to:
```
- pip:
    - dse-do-utils --no-index --find-links=/project_data/data_asset/dse_do_utils-0.3.0.0.tar.gz
```

Download the package as a wheel/zip (uncomment):

In [3]:
# !pip download dse-do-utils==0.3.0.0 -d /project_data/data_asset/

  File was already downloaded /project_data/data_asset/dse_do_utils-0.2.2.3.tar.gz
Successfully downloaded dse-do-utils


Make the file visible in the WS Data Assets UI:

In [ ]:
file_name = 'dse_do_utils-0.3.0.0.tar.gz'
file_path = '/project_data/data_asset/' + file_name
with open(file_path, 'rb') as f:
    from project_lib import Project
    project = Project.access()
    project.save_data(file_name=file_name, data=f, overwrite=True)

## 2. Installation as package in JupyterLab<a class="anchor" id="jl_install"></a>
If you want to make changes to the dse-do-utils, you can download the package folder and copy into a folder in JupyterLab.<br>
For instance in `/packages/python/`. <br>
Or pip install in the folder:
```
!pip install dse-do-utils==0.3.0.0 --target='/packages/python'
```

Then add this folder (in addition to a `scripts` folder) to the Python path by adding the following in a cell in a notebook:
```
import sys, os
for folder in ['packages/python', 'scripts']:
    path = os.path.join(os.environ['PWD'], folder)
    if path not in sys.path:
        sys.path.insert(0, path)
```

In [20]:
# !pip install dse-do-utils==0.3.0.0 --target='/packages/python'

In [4]:
import sys, os
for folder in ['packages/python', 'scripts']:
    path = os.path.join(os.environ['PWD'], folder)
    if path not in sys.path:
        sys.path.insert(0, path)

## 3. Installation as modules in Jupyter<a class="anchor" id="jupyter_install"></a>
In CPDv2.5 and working with Jupyter, there is no longer a `scripts` or `packages/python` directory.
The work-around to work with modules is to use a notebook to write a file to a local (hidden) file system.

Two options:
1. In a notebook, copy the contents of a module in a cell and add the following as the first line in the cell:
```
%%writefile mymodule.py
```
Run the cell. It writes a module `mymodule` in a location that is on the default Python path

2. Upload the modules of the dse-do-utils package as Data Assets.
Then run the something like following to write the contents as a file on the Python path.
```
from project_lib import Project
project = Project.access()
# Read module from data assets:
my_module = project.get_file(asset_name)
my_module.seek(0)
# Write module in 'current directory to allow import'
f = open(asset_name, 'wb')
f.write(my_module.read())
```

### 3.1 Jupyter Option 1 - Example of writing a module from a cell

In [5]:
%%writefile mymodule.py
class MyClassA():
    def __init__(self, name: str='myname'):
        self.name = name
        
    def get_name(self):
        return self.name

Writing mymodule.py


Import and use:

In [6]:
from mymodule import MyClassA
aa = MyClassA()
aa.get_name()

'myname'

As a side note, you can load the contents of the module back into a cell by using:
```
%load mymodule.py
```

Uncomment:

In [9]:
# %load mymodule.py

### 3.2 Jupyter Option 2 - Import module from file

1. Upload an existing module file as a data asset.
2. Create a notebook and using the Project APIs to read the file from assets and write as file in the 'current directory' (which is the same as option 1)

In [12]:
def cpd25_load_module_from_assets(asset_name: str):
    """Loads a Python module from the data assets of a CPD2.5 project and writes it as a file.
    So that the file is in the Python path of a notebook. 
    So that the module can be imported from a notebook.
    For use in CPDv2.5 icw regular notebooks (i.e. no JupyterLab) only
    
    Usage:
    1. Upload an existing Python module as a data asset
    2. Create a notebook
    3. Add this function definition
    4. In a cell, run a call to this function, e.g. `cpd25_load_module_from_assets('mymodule.py')`
    """
    from project_lib import Project
    project = Project.access()
    # Read module from data assets:
    my_module = project.get_file(asset_name)
    my_module.seek(0)
    # Write module in 'current directory to allow import'
    f = open(asset_name, 'wb')
    f.write(my_module.read())

In [13]:
def cpd25_load_all_modules_from_assets():
    """Loads all .py files in the data assets and writes them the 'disk' so that they can be imported from a notebook.
    """
    from project_lib import Project
    project = Project.access()
    for asset in project.get_assets():
        asset_name = asset['name']
        if asset_name[-3:] == ".py":
            print(f"Write module '{asset_name}'")
            cpd25_load_module_from_assets(asset_name)

Example of loading a single module (after it has been upoaded as Data Asset):

In [ ]:
# cpd25_load_module_from_assets('mymodule.py')

Automatically save all Python modules in Data Assets as files:

In [15]:
# cpd25_load_all_modules_from_assets()

## 4. Install/use as modules in the DO Model Builder<a class="anchor" id="mb_install"></a>
The challenge is that the environment of the DO Model Builder cannot be customized.<br>
But it does offer an option to manually add modules to the solver code.<br>
Modules can only be added from your local workstation.<br>
Unfortunately, the modules will need to be re-upoaded any time the solver model is re-initialized.


Steps:
1. Download the modules to your local workstation. Select the ones that are required by the solver code. Typically `optimizationengine.py`, `datamanager.py` and `scenariomanager.py` (which is required by optimizationengine)
2. After updating the code of the Model Builder, add the modules. You can select a set of modules and upload simultaniously.

In this usage, the package has been deconstructed as a set of modules

# Usage
How to use/import the definitions in the package.

There are 3 ways:
1. Import from the dse_do_utils package
2. Import from a module within the package
3. Import from a module as part of a set of modules

# 5. Import from the dse_do_utils package<a class="anchor" id="package_import"></a>
The `__init__.py` maps definitions from the various internal modules to the package level.<br>
This allows import directly from the package. For instance:
```
from dse_do_utils import ScenarioManager
```

This works when:
* The package has been installed as a whole (either throught the customized environment or as a directory in JupyterLab)
* No need to make changes to the code

In [17]:
# from dse_do_utils import ScenarioManager

# 6. Import from the dse_do_utils modules within the package<a class="anchor" id="submodule_import"></a>
Directly import the definitions from the internal modules. For instance:
```
from dse_do_utils.scenariomanager import ScenarioManager
```

This approach is necessary when actively making changes to the code of the package.
After a change, the module needs to me reloaded:
1. Either explicitly using reload:
```
import imp, dse_do_utils
imp.reload(dse_do_utils.scenariomanager)
```
2. Or indirectly via autoreload jupyter extension:
```
%load_ext autoreload
%autoreload 2
```
Both autoreload and imp.reload will reload the internal module. But that will only update the definition if it was directly imported from the module itself (and not indirectly via the `__init__.py` at the package level). 

In [18]:
# from dse_do_utils.scenariomanager import ScenarioManager

# 7. Import from a module as part of a set of modules<a class="anchor" id="module_import"></a>
In case the package was extracted into a set of individual modules, like when attaching the modules to a DO model, we need to import directly from the stand-alone module:
```
from scenariomanager import ScenarioManager
```

This would apply to a solver notebook. But when testing the notebook, it needs to load from the installed package, we need to be flexible. First try to import from the package. if that fails, import from the module:
```
try:
    from dse_do_utils.datamanager import DataManager
    from dse_do_utils.optimizationengine import OptimizationEngine
except:
    from datamanager import DataManager
    from optimizationengine import OptimizationEngine
```

In [19]:
# try:
#     from dse_do_utils.datamanager import DataManager
#     from dse_do_utils.optimizationengine import OptimizationEngine
# except:
#     from datamanager import DataManager
#     from optimizationengine import OptimizationEngine